UNSW-NB15 数据集 BT-TWD 与基线实验

本 notebook 按步骤运行：环境准备 → 数据加载 → 特征构建 → 单次留出与 K 折实验 → 输出检查 → 可选扩展。

本 notebook 结构与 03_bank_bttwd_main.ipynb 保持一致，仅更换 UNSW 配置，并适配 YAML 中 train/test 双文件的读取。

In [ ]:
# 步骤0：环境与依赖
import os
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

root_path = Path(os.getcwd()).resolve()
while root_path.name != "BT-TWD" and root_path.parent != root_path:
    root_path = root_path.parent

if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    run_holdout_experiment,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_name = "unsw_nb15.yaml"
cfg_path = root_path / "configs" / cfg_name
cfg = load_yaml_cfg(str(cfg_path))

global_seed = cfg.get("SEED", {}).get("global_seed", 42)
set_global_seed(global_seed)

log_info("【步骤0摘要】环境与依赖加载完毕，项目根目录设置为: {}".format(root_path))
print("当前使用配置文件:", cfg_path)
show_cfg(cfg)


In [ ]:
# 步骤1：加载 UNSW 训练集原始数据
log_info("【步骤1】开始加载 UNSW 训练集原始数据……")

df_raw, target_col = load_dataset(cfg)

if "split" in df_raw.columns:
    df_train = df_raw[df_raw["split"] == "train"].copy()
    df_test = df_raw[df_raw["split"] == "test"].copy()
    log_info(
        f"【步骤1】检测到 train/test 双文件：训练样本数={len(df_train)}，测试样本数={len(df_test)}，总计={len(df_raw)}。"
    )
else:
    df_train = df_raw.copy()
    df_test = None
    log_info(f"【步骤1】训练集加载完成，样本数={len(df_train)}, 目标列='{target_col}'。")

log_info(f"【步骤1】目标列='{target_col}'，将使用训练集样本进行后续实验。")

display(df_train.head())
display(df_train[target_col].value_counts())
display(df_train[target_col].value_counts(normalize=True))

if df_test is not None:
    log_info("【步骤1】测试集标签分布预览（仅供参考，不参与训练）。")
    display(df_test[target_col].value_counts())
    display(df_test[target_col].value_counts(normalize=True))

log_info("【步骤1摘要】UNSW 训练数据加载完成，已输出前 5 行与标签分布。")

In [ ]:
# 步骤2：特征构建 & 拆分 X / y / 桶特征
log_info("【步骤2】开始根据 YAML 配置构建特征，并拆分为 X / y / 桶特征……")

X_train, y_train, bucket_df_train = prepare_features_and_labels(
    df_train, cfg
)

log_info(
    f"【步骤2】特征构建完成，X_train 形状={X_train.shape}，"
    f"y_train 长度={len(y_train)}，bucket_df_train 形状={bucket_df_train.shape}。"
)
display(bucket_df_train.head())
log_info("【步骤2摘要】UNSW 训练集特征构建完成，后续将基于该特征进行 BT-TWD 与基线实验。")

In [ ]:
# 步骤3：单次留出验证（BT-TWD + 各基线）
log_info("【步骤3】开始执行单次留出验证实验（BT-TWD + 基线）……")

holdout_metrics = run_holdout_experiment(
    X_train,
    y_train,
    bucket_df_train,
    cfg,
)
holdout_df = pd.DataFrame(holdout_metrics)
display(holdout_df)

log_info("【步骤3摘要】单次留出验证完成，已输出各模型的指标表。")


In [ ]:
# 步骤4：K 折交叉验证（BT-TWD + 各基线）
log_info("【步骤4】开始执行 K 折交叉验证实验（BT-TWD + 基线）……")

results = run_kfold_experiments(
    X_train,
    y_train,
    bucket_df_train,
    cfg,
)
metrics_df = pd.DataFrame(results)
display(metrics_df)

log_info("【步骤4摘要】K 折交叉验证完成，已展示每个模型在各折上的指标。")


In [ ]:
# 步骤5：检查输出目录（结果 / 图像）
log_info("【步骤5】检查输出目录，确认结果文件是否已保存……")

output_cfg = cfg.get("OUTPUT", {})
results_dir = root_path / output_cfg.get("results_dir", "notebooks/results")
figs_dir = root_path / output_cfg.get("figs_dir", "notebooks/figs")

print("结果目录:", results_dir)
print("图像目录:", figs_dir)

if results_dir.exists():
    print("结果文件：", os.listdir(results_dir))
else:
    print("结果目录不存在。")

if figs_dir.exists():
    print("图像文件：", os.listdir(figs_dir))
else:
    print("图像目录不存在。")

log_info("【步骤5摘要】输出目录检查完成，请根据需要查看 CSV/图像文件。")


In [ ]:
# 步骤6：基于 K 折结果的整体汇总（可选）
log_info("【步骤6】基于 K 折结果做整体指标汇总……")

if 'metrics_df' in locals() and isinstance(metrics_df, pd.DataFrame) and not metrics_df.empty:
    if 'model' in metrics_df.columns:
        aggregated_df = metrics_df.groupby('model').mean(numeric_only=True).reset_index()
        display(aggregated_df)
    else:
        log_info("【步骤6】metrics_df 中未找到 'model' 列，跳过聚合。")
else:
    log_info("【步骤6】暂无可汇总的交叉验证结果，跳过。")

log_info("【步骤6摘要】整体指标汇总完成（若有可用数据）。")


In [ ]:
# 步骤7：XGB + TWD 全局阈值搜索（UNSW 专用，可选）
try:
    from bttwdlib import run_unsw_xgb_twd_experiment  # type: ignore
    has_unsw_xgb_twd = True
except ImportError:
    has_unsw_xgb_twd = False

if has_unsw_xgb_twd:
    log_info("【步骤7】开始运行 XGB + TWD 基线实验（含全局阈值搜索）……")
    best_alpha, best_beta, best_stats = run_unsw_xgb_twd_experiment(cfg)
    log_info(
        f"【步骤7摘要】XGB + TWD 全局阈值搜索完成："
        f"alpha={best_alpha:.3f}, beta={best_beta:.3f}, "
        f"Regret={best_stats.get('Regret', best_stats.get('regret', float('nan'))):.4f}"
    )
else:
    log_info("【步骤7摘要】未检测到 run_unsw_xgb_twd_experiment 函数，跳过该实验。")
